# linear regression

In [ ]:
import pandas as pd
import matplotlib.pyplot as pyt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("Tech_sector_diversity_demographics_2016.csv")
df.head()

NameError: name 'pd' is not defined